In [1]:
###倒入會用到的函式庫
from PIL import Image 
import csv
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import math

In [2]:
###把"鴨子樣本"的RGB存成.csv檔
#其label皆為1
img = Image.open("C:\\Users\\rachel\\Desktop\\UseBayesToFindingDuck\\image\\duck.jpg") 
img_width = 255
img_height = 255 
img_width, img_height, = img.size #我想要的圖片大小
pix = img.load() # 讀取像素值

##創造一個csv檔來儲存讀取圖片中每個像素的資料
#W+可讀可寫可建立
with open("C:\\Users\\rachel\\Desktop\\UseBayesToFindingDuck\\pixel_RGBtrain.csv", 'w+') as f:  
    
    f.write('r,g,b,label\n') #在第一列的地方標註其每欄的值為何 (分別是R ,G ,B ,Label)
    for x in range(img_width):           #for loop x as img_width 
        for y in range(img_height):      #for loop y as img_height
            r = pix[x,y][0]                  
            g = pix[x,y][1]                  
            b = pix[x,y][2]                  
            f.write('{0},{1},{2},1\n'.format(r,g,b,1))     #format as r,g,b,1


In [3]:
###把"非鴨子樣本"的RGB存成.csv檔
#其label皆為2
imgb = Image.open("C:\\Users\\rachel\\Desktop\\UseBayesToFindingDuck\\image\\unduck.jpg")

imgb_width = 255
imgb_height = 255 
imgb_width, imgb_height, = imgb.size #Size of the image I want
pixb = imgb.load() # loading the pixel value

#a+可讀可寫可建立不覆蓋可追加
with open("C:\\Users\\rachel\\Desktop\\UseBayesToFindingDuck\\pixel_RGBtrain.csv", 'a+') as fs:
                                            
    for x in range(imgb_width):          
        for y in range(imgb_height):       
            r = pixb[x,y][0]                 
            g = pixb[x,y][1]                  
            b = pixb[x,y][2]                  
            fs.write('{0},{1},{2},2\n'.format(r,g,b,2))    


In [10]:
###train_data準備:
#讀取csv檔，並將它轉成dataset
lines = csv.reader(open("C:\\Users\\rachel\\Desktop\\UseBayesToFindingDuck\\pixel_RGBtrain.csv", "r"))
dataset = list(lines)  # 每行存進去一個list
    # print(dataset)
for i in range(1, len(dataset)):  # 因為第0行為標註其每欄的值，type式字串，無法轉成數字，所以從1開始
    dataset[i] = [float(x) for x in dataset[i]]  # 將轉成數字的Data存進去dataset
dataset = dataset[1:]  # 忽略標題那一欄  

###test_data準備:
#讀取我們test_data_img的圖
test_data_img = Image.open("C:\\Users\\rachel\\Desktop\\UseBayesToFindingDuck\\image\\full_duck_down.jpg")
test_data_colourPixels = test_data_img.convert("RGB")
test_data_colourArray = np.array(test_data_colourPixels.getdata()).reshape(test_data_img.size + (3,))
test_data_indicesArray = np.moveaxis(np.indices(test_data_img.size), 0, 2)#將數組的軸移到新位置。

allArray = np.dstack((test_data_indicesArray, test_data_colourArray)).reshape((-1, 5))
df = pd.DataFrame(allArray, columns=["x", "y", "red", "green", "blue"])
df.drop(['x', 'y'], axis=1, inplace=True) #將"X"，"Y"刪除
testdata = df.to_numpy().astype(float).tolist()


   


In [11]:
###將R,G,B這三個數值欄append成兩個label的vector
###計算兩個label的vector各自的均值與標準差

def mean(numbers): #均值
    return sum(numbers) / float(len(numbers))  # Average number


def SD(numbers):  # 標準差
    avg = mean(numbers)
    variance = sum([pow(x - avg, 2) for x in numbers]) / float(len(numbers) - 1)
    return math.sqrt(variance)
def calculate(dataset):  # 計算每個屬性的均值和標準差
    summaries = [(mean(virtue), SD(virtue)) for virtue in
                 zip(*dataset[1:-2])]  # zip函數將資料樣本按照屬性分組為一個個清單，然後可以對每個屬性計算均值和標準差
    del summaries[-1]
    return summaries 

separated = {} # 最後一個屬性（-1）為label，返回一個label到dataset做mappimg。
for i in range(len(dataset)):
    vector = dataset[i]
    if (vector[-1] not in separated):  # [-1] 代表array倒數第一個值
        separated[vector[-1]] = []
    separated[vector[-1]].append(vector)

summaries = {}
for classValue, instances in separated.items():
    summaries[classValue] = calculate(instances)

In [12]:
###計算各個類別的Gaussian probabilistic likelihood models，可以得到各個label的條件機率
###最後再使用Bayes Classifier來做prediction

def Probability(x, mean, SD):  # Gaussian probabilistic likelihood models
    exponent = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(SD, 2)))) 
    return (1 / (math.sqrt(2 * math.pi) * SD)) * exponent


predictions = []
for i in range(len(testdata)):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for j in range(len(classSummaries)):
            mean, SD = classSummaries[j]
            x = testdata[i][j]
            probabilities[classValue] *= Probability(x, mean, SD)
            
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    predictions.append(bestLabel)

In [13]:
###GenerateOutputImage
###將預測的結果做可視化處理
outputImages =[]  
for i in range(len(predictions)):
    if(1 == int(predictions[i])):#預測的結果為1，代表預測的類別為鴨子，則將當個像素點設為白色
        outputImages.append([255,255,255])
    else:#其他的預測結果，皆代表預測的類別為非鴨子，則將當個像素點設為黑色
        outputImages.append(([0,0,0]))
outputImages = np.array(outputImages)


#將可視化結果圖的大小設成跟原始圖片一樣的大小
original_img = cv2.imread("C:\\Users\\rachel\\Desktop\\UseBayesToFindingDuck\\image\\full_duck_down.jpg")
size = original_img.shape 
array = np.reshape(outputImages, (size[0], -1))

# 利用 Pillow 用上面的新array創造一個新的圖片
output_image = Image.fromarray(array)
output_image = output_image.resize((size[0], size[1]))
if output_image.mode == 'P' or output_image.mode == 'RGBA' or output_image.mode == 'I':
    output_image = output_image.convert('RGB')

output_image.save("C:\\Users\\rachel\\Desktop\\UseBayesToFindingDuck\\Output\\Output_full_duck_down.jpg")
print("預測結束並以輸出結果圖")

預測結束並以輸出結果圖


In [9]:
###將兩張圖合併成大圖
##因為原始圖太大，所以將圖片拆成上下兩部分，分別取名為full_duck_1跟full_duck_2
##需跑兩次的4-9cell 來產出Output_full_duck_1跟Output_full_duck_2
full_duck_up_img = cv2.imread("C:\\Users\\rachel\\Desktop\\UseBayesToFindingDuck\\Output\\Output_full_duck_up.jpg")
full_duck_down_img = cv2.imread("C:\\Users\\rachel\\Desktop\\UseBayesToFindingDuck\\Output\\Output_full_duck_down.jpg")

full_duck_up_img_gray = cv2.cvtColor(full_duck_up_img, cv2.COLOR_BGR2GRAY)
full_duck_down_img_gray = cv2.cvtColor(full_duck_down_img, cv2.COLOR_BGR2GRAY)

full_duck = np.concatenate((full_duck_up_img_gray, full_duck_down_img_gray)) 
cv2.imwrite("C:\\Users\\rachel\\Desktop\\UseBayesToFindingDuck\\Output\\Output_full_duck.jpg",full_duck)

True